# 🌍 ISB Translation (Clean Version)
This notebook translates ISB scraped data from German to English using DeepL API.

In [ ]:

import pandas as pd
import requests
import time


In [ ]:

DEEPL_API_KEY = "your-api-key-here"  # Replace this with your DeepL API key
DEEPL_URL = "https://api-free.deepl.com/v2/translate"


In [ ]:

def translate_text(text, target_lang="EN"):
    if pd.isna(text) or not isinstance(text, str) or not text.strip():
        return text
    for _ in range(3):
        try:
            response = requests.post(
                DEEPL_URL,
                data={"auth_key": DEEPL_API_KEY, "text": text, "target_lang": target_lang},
                timeout=10
            )
            if response.status_code == 429:
                print("⏳ Rate limit hit. Waiting...")
                time.sleep(10)
                continue
            if response.status_code != 200:
                print(f"❌ Error {response.status_code}: {text[:50]}")
                return text
            return response.json()["translations"][0]["text"]
        except Exception as e:
            print(f"⚠️ Retry error: {e}")
            time.sleep(5)
    return text


In [ ]:

# Load German scraped CSV
df = pd.read_csv("funding-isb.csv")


In [ ]:

# Translate main columns
translated_df = df.copy()
for col in ["name", "description", "eligibility", "amount", "procedure"]:
    print(f"🔤 Translating column: {col}")
    translated_df[col] = df[col].apply(translate_text)
    time.sleep(2)


In [ ]:

# Translate only the contact role (before the first '|')
def translate_contact_role(contact):
    if pd.isna(contact) or not isinstance(contact, str) or " | " not in contact:
        return contact
    try:
        role, phone, email = contact.split(" | ", 2)
        role_translated = translate_text(role)
        return f"{role_translated} | {phone} | {email}"
    except:
        return contact


In [ ]:

print("🔤 Translating contact roles...")
translated_df["contact"] = df["contact"].apply(translate_contact_role)


In [ ]:

# Save translated output
translated_df.to_csv("funding-isb-english.csv", index=False)
print("✅ Translation complete. File saved as funding-isb-english.csv")
